In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50,VGG19,VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set paths and parameters
data_path = "DataSets/AirCrafts"#'crop'
image_size = (224, 224)  # Resize images to match model input size
batch_size = 32

# Create ImageDataGenerator instance
datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)

# Load training and validation sets
train_data = datagen.flow_from_directory(
    directory=data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    directory=data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 25565 images belonging to 74 classes.
Found 6352 images belonging to 74 classes.


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import tensorflow as tf

# Load base model and freeze layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_data.class_indices), activation='softmax')  # Output layer for classification
])

# Compile model with frozen base
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Initial training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Evaluate on validation data
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy after initial training: {val_acc:.2f}")

# Fine-tuning the last 10 layers of the base model
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Re-compile the model with a low learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning phase
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Evaluate on validation data after fine-tuning
val_loss_fine, val_acc_fine = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {val_acc_fine:.2f}")

# Save the model
model.save('fighter_jet_classifier.keras', save_format='keras')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


D:\AnnA\envs\diat\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 1696s 2s/step - accuracy: 0.0304 - loss: 4.4555 - val_accuracy: 0.0461 - val_loss: 4.0422
Epoch 2/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 977s 1s/step - accuracy: 0.0408 - loss: 4.1774 - val_accuracy: 0.0554 - val_loss: 4.0315
Epoch 3/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 979s 1s/step - accuracy: 0.0447 - loss: 4.1209 - val_accuracy: 0.0529 - val_loss: 4.0260
Epoch 4/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 983s 1s/step - accuracy: 0.0487 - loss: 4.0868 - val_accuracy: 0.0559 - val_loss: 4.0224
Epoch 5/10
799/799 ━━━━━━━━━━━━━━━━━━━━ 0s 945ms/step - accuracy: 0.0538 - loss: 4.0519

KeyboardInterrupt: 

In [4]:

# Plot training and validation accuracy and loss
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Plot accuracy
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)



NameError: name 'history' is not defined

In [ ]:
# Prediction function for a single image
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image_class(model, img_path, class_indices):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions[0])

    # Convert index to class name
    class_labels = {v: k for k, v in class_indices.items()}
    predicted_class_name = class_labels[predicted_class_idx]

    print(f"Predicted class: {predicted_class_name}")
    return predicted_class_name

# Example usage
img_path = 'path/to/your/test_image.jpg'  # Replace with your test image path
predicted_class = predict_image_class(model, img_path, train_data.class_indices)
